<a href="https://colab.research.google.com/github/Hoale2908/Facebook_scraper/blob/main/FB_Crawler_Hoa_Le.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Preparation
- Using Chrome web browser
- Install the Chrome extension "Get cookies.txt" (can be found [here](https://chrome.google.com/webstore/detail/get-cookiestxt/bgaddhkoddajcdgocldbbfleckgcbcid?hl=en)).
-  Sign in your account on Facebook using Chrome. 
- Click on the extension and then "Extract" to save the .txt file to your computer.

# 1. Install Library

Run this block and wait (about 1 min) for the installation to finish. 

In [ ]:
#@title
%pip install facebook_scraper
from pathlib import Path
print("Setup completed.")

# 2. Set up working folder and Upload cookies

Upload the file .txt that you have downloaded using the "Get cookies.txt" extension.

In [ ]:
#@title
# Create a folder
FOLDER_NAME = "Facebook_scraping"
FOLDER_PATH = "/content/" + FOLDER_NAME + "/"

import os 
os.mkdir(FOLDER_PATH)

# Upload the cookies
os.chdir(FOLDER_PATH)
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Get path to cookies
COOKIE_PATH = FOLDER_PATH + (os.listdir(FOLDER_PATH)[0])

print('\n \n Completed.')

# 3. Provide inputs

Open the Facebook page on the web browser. Copy the remaining part of the URL starting after the "facebook.com/". 

For example: The full URL for CNN's page is "https://www.facebook.com/cnn", copy the string "cnn".

In [ ]:
#@title 
# input the Page ID
FANPAGE_LINK = input("Handle of the target page:")

# estimate the number of posts, then divide it by 4
no_of_post = input('Please estimate the number of posts to be scraped:')
PAGES_NUMBER = round(int(no_of_post) / 4)

# 4. Scrape the data

Start scraping the data. When finished, all posts on the page will be saved to an Excel file named "All_posts.xlsx" and downloaded to your computer (in the default folder for downloads).

In [ ]:
#@title
from facebook_scraper import get_posts
import pandas as pd

post_list = []
for post in get_posts(FANPAGE_LINK, 
                    options={"comments": True, "reactions": True, "allow_extra_requests": True}, 
                    extra_info=True, pages=PAGES_NUMBER, cookies=COOKIE_PATH):
    post_list.append(post)

# Export all posts on the page to an Excel file
df_allpost = pd.DataFrame(post_list)
df_allpost.to_excel("All_posts.xlsx")
files.download("All_posts.xlsx")

After the file is generated and downloaded to your computer, open it and find the ID of the post that you want to scrape the comments. You will need to do this for each post that you want to scrape comments. 

After running the below code, the comments will be saved to an Excel file ended with "_Comments_on_post" and prefixed by the post ID.

In [ ]:
#@title 
# Get the post ID
target_post_id = input("ID of the post that you want to scrape its comments (you can find it in the file All_posts.xlsx):")

# Get the full comments of such post
index_row = df_allpost.index[df_allpost['post_id'] == target_post_id].tolist()[0]
target_post_comment = df_allpost.loc[index_row,"comments_full"]

# Export all the comments to the target post
df_target_post_comment = pd.DataFrame(target_post_comment)
df_target_post_comment.to_excel(target_post_id + "_comments_on_post.xlsx")
files.download(target_post_id + "_comments_on_post.xlsx")

Now as all the comments on the post have been scraped, you can continue to scrape all the replies to each comment if you wish to by running the below code. Replies will be saved to separate Excel files with the Comment ID on the filename. E.g. Replies to 10000222222.xlsx

In [ ]:
#@title
# For each comment that has replies, we need to create a df for the replies
# Iterating thru the comments of the target post; check if it has replies, if yes: create a df of the replies and save to an Excel file.
for c in target_post_comment:
    if (len(c['replies'])) > 0:
        rep_df = pd.DataFrame(c['replies'])      
        rep_df.to_excel(c['comment_id'] + "_replies_to_comment.xlsx")
        files.download(c['comment_id'] + "_replies_to_comment.xlsx")